In [477]:
import pickle
import numpy as np
from sklearn import preprocessing
import pandas as pd

In [478]:
results_n_pas = []
results_n_clientes = []
results_var_demanda_pas = []
results_dist_clientes = []

for i in range(10):
    path = 'Ex2/PW/dataframe_' + str(i + 1) + '.pkl'
    print(path)
    with open(path, 'rb') as handle:
        b = pickle.load(handle)
        results_n_pas.append(np.sum(b['ap']))
        results_n_clientes.append(np.sum(b['acp']))
        results_var_demanda_pas.append(np.var(np.matmul(b['acp'].transpose(), b['cc'])))
        results_dist_clientes.append(np.sum(b['acp']*b['d']))

Ex2/PW/dataframe_1.pkl
Ex2/PW/dataframe_2.pkl
Ex2/PW/dataframe_3.pkl
Ex2/PW/dataframe_4.pkl
Ex2/PW/dataframe_5.pkl
Ex2/PW/dataframe_6.pkl
Ex2/PW/dataframe_7.pkl
Ex2/PW/dataframe_8.pkl
Ex2/PW/dataframe_9.pkl
Ex2/PW/dataframe_10.pkl


In [479]:
print(results_n_pas)
print(results_n_clientes)
print(results_var_demanda_pas)
print(results_dist_clientes)

[29.0, 27.0, 87.0, 49.0, 29.0, 81.0, 35.0, 47.0, 25.0, 17.0]
[191.0, 207.0, 192.0, 177.0, 190.0, 204.0, 206.0, 216.0, 175.0, 225.0]
[0.8260763601528387, 0.589030272631835, 0.38156231642988325, 0.5196552733064596, 0.6612605552377678, 0.44986295871702536, 0.710452015217133, 0.5181940580074524, 0.5435360524491136, 1.1206428061454263]
[48732.326419452205, 61001.92296847254, 49922.96891197494, 43922.229028932634, 52119.78156566477, 55239.45609224751, 52648.398940217456, 55078.32952274348, 46988.90239773596, 60196.12213963527]


In [480]:
# Criação das matrizes dos critérios
def generate_criteria_matrix(data):
    m_result = np.eye(len(data))
    length = range(len(data))
    for i in length:
        for j in length:
            m_result[i][j] = data[j]/data[i]
    return m_result

In [481]:
# criação da matriz do critério 'número de PAs' 
n_pas = generate_criteria_matrix(results_n_pas)
# criação da matriz do critério 'número de clientes atendidos' 
n_clientes = generate_criteria_matrix(results_n_clientes)
# criação da matriz do critério 'variância da demanda dos PAs' 
var_demanda_pas = generate_criteria_matrix(results_var_demanda_pas)
# criação da matriz do critério 'distância total dos clientes' 
dist_clientes = generate_criteria_matrix(results_dist_clientes)

In [482]:
pd.DataFrame(n_pas)

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.931034,3.000000,1.689655,1.000000,2.793103,1.206897,1.620690,0.862069,0.586207
1,1.074074,1.000000,3.222222,1.814815,1.074074,3.000000,1.296296,1.740741,0.925926,0.629630
2,0.333333,0.310345,1.000000,0.563218,0.333333,0.931034,0.402299,0.540230,0.287356,0.195402
3,0.591837,0.551020,1.775510,1.000000,0.591837,1.653061,0.714286,0.959184,0.510204,0.346939
4,1.000000,0.931034,3.000000,1.689655,1.000000,2.793103,1.206897,1.620690,0.862069,0.586207
5,0.358025,0.333333,1.074074,0.604938,0.358025,1.000000,0.432099,0.580247,0.308642,0.209877
6,0.828571,0.771429,2.485714,1.400000,0.828571,2.314286,1.000000,1.342857,0.714286,0.485714
7,0.617021,0.574468,1.851064,1.042553,0.617021,1.723404,0.744681,1.000000,0.531915,0.361702
8,1.160000,1.080000,3.480000,1.960000,1.160000,3.240000,1.400000,1.880000,1.000000,0.680000
9,1.705882,1.588235,5.117647,2.882353,1.705882,4.764706,2.058824,2.764706,1.470588,1.000000


In [483]:
# normalizando de acordo com os critérios de decisão (1 - 9) - max e min globais
def normalize_one_nine(data):
    normalized_data = np.zeros(shape=data.shape)
    min_ = data.min()
    max_ = data.max()
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            normalized_data[i][j] = (8*((data[i][j] - min_)/(max_ - min_)) + 1).round(0)
    return normalized_data

In [484]:
normalized_n_pas = normalize_one_nine(n_pas) 
normalized_n_clientes = normalize_one_nine(n_clientes) 
normalized_var_demanda_pas = normalize_one_nine(var_demanda_pas) 
normalized_dist_clientes = normalize_one_nine(dist_clientes)

In [485]:
# normalizando somente as colunas (0 - 1): parte do processo das prioridades
def normalize_columnwise(data):
    normalized_data = np.zeros(shape=data.shape)
    for i in range(data.shape[1]):
        min_ = data[:,i].min()
        max_ = data[:,i].max()
        for j in range(data.shape[0]):
            normalized_data[j][i] = ((data[j][i] - min_)/(max_ - min_))
    return normalized_data

# gera vetor de prioridades de uma matriz
def get_priorities(data):
    normalized_data = normalize_columnwise(data)
    priorities = []
    for row in range(normalized_data.shape[0]):
        priorities.append(np.mean(normalized_data[row]))
    return priorities/sum(priorities)

In [486]:
sum(get_priorities(normalized_n_pas))

0.9999999999999998

In [487]:
d = {'prioridades_n_pas': get_priorities(normalized_n_pas),
     'prioridades_n_clientes': get_priorities(normalized_n_clientes),
     'prioridades_var_demanda_pas': get_priorities(normalized_var_demanda_pas),
     'prioridades_dist_clientes': get_priorities(normalized_dist_clientes) }

In [488]:
df_prioridades = pd.DataFrame(d)
df_prioridades

,prioridades_n_pas,prioridades_n_clientes,prioridades_var_demanda_pas,prioridades_dist_clientes
0,0.123908,0.129599,0.039130,0.144444
1,0.143013,0.054359,0.098328,0.000000
2,0.000000,0.114683,0.200669,0.138697
3,0.063865,0.193901,0.119064,0.229885
4,0.123908,0.134571,0.069900,0.091188
5,0.000000,0.069274,0.168227,0.063602
6,0.086245,0.059330,0.069900,0.085441
7,0.063865,0.045409,0.119064,0.063602
8,0.165939,0.198873,0.115719,0.178544
9,0.229258,0.000000,0.000000,0.004598


In [544]:
# prioridades entre critérios
# Rows:
#   0 - nº PAs
#   1 - Distância Total
#   2 - Variância Demanda PAs
#   3 - Clientes Atendidos

# impotancia_criterios = np.array([[1,5,7,0.143], [0.2,1,7,0.125], [0.143, 0.143, 1, 0.111], [7,8,9,1]])
impotancia_criterios = np.array([[1,5,7, 1/7], [1/5,1,7,1/8], [1/7, 1/7, 1, 1/9], [7,8,9,1]]).round(3)

In [545]:
pd.DataFrame(impotancia_criterios)

,0,1,2,3
0,1.000,5.000,7.0,0.143
1,0.200,1.000,7.0,0.125
2,0.143,0.143,1.0,0.111
3,7.000,8.000,9.0,1.000


In [528]:
# Cálculo do índice de consistência
(np.linalg.eig(impotancia_criterios)[0].max() - 4)/3

(0.24234374664129277+0j)

In [491]:
prioridades_criterios = get_priorities(impotancia_criterios)
prioridades_criterios

array([0.23847217, 0.13772549, 0.        , 0.62380234])

In [493]:
def calc_final_score(p1, p2):
    result = p1.copy()
    result['Geral'] = np.zeros(p1.shape[0])
    for row in range(p1.shape[0]):
        result['Geral'][row] = np.sum(p1.loc[row] * p2)
    return result

In [529]:
final = calc_final_score(df_prioridades, prioridades_criterios)

In [530]:
final

,prioridades_n_pas,prioridades_n_clientes,prioridades_var_demanda_pas,prioridades_dist_clientes,Geral
0,0.123908,0.129599,0.039130,0.144444,0.137503
1,0.143013,0.054359,0.098328,0.000000,0.041591
2,0.000000,0.114683,0.200669,0.138697,0.102315
3,0.063865,0.193901,0.119064,0.229885,0.185338
4,0.123908,0.134571,0.069900,0.091188,0.104966
5,0.000000,0.069274,0.168227,0.063602,0.049216
6,0.086245,0.059330,0.069900,0.085441,0.082036
7,0.063865,0.045409,0.119064,0.063602,0.061159
8,0.165939,0.198873,0.115719,0.178544,0.178338
9,0.229258,0.000000,0.000000,0.004598,0.057540


In [538]:
final['prioridades_n_clientes']

0    0.129599
1    0.054359
2    0.114683
3    0.193901
4    0.134571
5    0.069274
6    0.059330
7    0.045409
8    0.198873
9    0.000000
Name: prioridades_n_clientes, dtype: float64